In [1]:
!pip install -q -U kaggle_environments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns


from kaggle_environments import make, evaluate, utils

# Задача
Описать поведение 12 различных агентов, играющих друг с другом в игру "Камень-ножницы-бумага".

# Специальные обозначения:
  - 0 – "камень"
  - 1 — "бумага"
  - 2 — "ножницы"

# Агенты встроенные в библиотеку
Данных агентов мы прописывать не будем:
  1. rock — всегда показывает камень
  2. paper — всегда показывает бумагу
  3. scissors – всегда показывает ножницы
  4. copy_opponent – копирует последний ход соперника
  5. reactionary – Если выиграл или ничья покажет знак который победил соперника в предыдущем раунде.
  6. counter_reactionary – в случае своей победы в предыдущем раунде показывает знак, который победил бы его знак в предыдущем раунде, а если проиграет покажет знак, который победил бы его соперника в прошлом раунде.
  7. statistical – реализует стратегию, основанную на статистическом анализе действий противника.

## Случайный агент

Выбирает одно из трех действий (камень, ножницы или бумага) случайным образом в каждом раунде.

In [3]:
def random_agent(observation, configuration):
  return random.choice([0,1,2])

# Цикличный агент

Выбирает камень в первом раунде, ножницы во втором, бумагу в третьем и затем повторяет этот цикл

In [4]:
def cycle_agent(observation, configuration):
  # Список ходов в цикле: камень, ножницы, бумага
    moves = [0, 1, 2]

    # Номер текущего раунда
    round_number = observation.step

    # Определение хода на основе номера раунда
    move_index = round_number % len(moves)

    return moves[move_index]

# Агент случайно предрасположенности №1

Показыает случайный знак с с весами камень - 0.5, бумага - 0.25, ножницы - 0.25

In [5]:
def random_pick_agent1(observation, configuration):
  # Массив значений
  arr = [0, 1, 2]
  # Веса для каждого значения (соответствуют вероятностям)
  weights = [0.5, 0.25, 0.25]
  # Выбор с учетом указанных весов
  result = random.choices(arr, weights=weights, k=1)[0]
  return result

# Агент случайно предрасположенности №2

Показыает случайный знак с с весами камень - 0.25, бумага - 0.5, ножницы - 0.25

In [6]:
def random_pick_agent2(observation, configuration):
  # Массив значений
  arr = [0, 1, 2]
  # Веса для каждого значения (соответствуют вероятностям)
  weights = [0.25, 0.5, 0.25]
  # Выбор с учетом указанных весов
  result = random.choices(arr, weights=weights, k=1)[0]
  return result

# Агент случайно предрасположенности №3

Показыает случайный знак с с весами камень - 0.25, бумага - 0.25, ножницы - 0.5

In [7]:
def random_pick_agent3(observation, configuration):
  # Массив значений
  arr = [0, 1, 2]
  # Веса для каждого значения (соответствуют вероятностям)
  weights = [0.25, 0.25, 0.5]
  # Выбор с учетом указанных весов
  result = random.choices(arr, weights=weights, k=1)[0]
  return result

# Турнир

Турнир проводиться между 12 агентами c количеством шагов – 100.

# Реализация турнира

In [14]:
def sum_of_winner_points(agent, list_of_opponents):
    sum_of_points = 0
    points = []
    for i in list_of_opponents:
        # Создаем среду "камень, ножницы, бумага"
        env = make("rps", configuration={"episodeSteps": 100}, debug=True)
        # Запускаем сражение
        env.run([agent, i])
        # Получаем счет одного агента
        points.append(env.steps[-1][0]['reward'])
    # Суммируем все очки агента после того как он сыграл со всеми агентами
    sum_of_points = sum(points)
    return sum_of_points

agents_list = ['rock', 'paper', 'scissors',
               'copy_opponent', 'reactionary',
               'counter_reactionary', 'statistical', random_agent,
               cycle_agent, random_pick_agent1, random_pick_agent2, random_pick_agent3]

score = {}

def tournament(agents_list):
    for i in agents_list:
        # Создаем копию списка, исключив текущего агента i
        list_without_i = [agent for agent in agents_list if agent != i]
        # Добавялем в список сууму очков агента
        score[i] = sum_of_winner_points(i, list_without_i)
    # Находим максимум очков из этого списка
    max_key = max(score, key=score.get)
    return f'Победитель турнира агент {max_key} со счетом {score[max_key]}'

print(tournament(agents_list))

Победитель турнира агент statistical со счетом 320.0


# Вывод
Я описал поведение 12 агентов и выяснил, что самым эффективным из них является агент statistical.